In [ ]:
# @title 1. Setup and Installation (Robust Version)
# Clone repository and install dependencies with error handling

# Remove existing directory and clear cache
!rm -rf Chatterbox-TTS-Server
!pip cache purge

# Clone repository
!git clone https://github.com/devnen/Chatterbox-TTS-Server.git
%cd Chatterbox-TTS-Server

print("✅ Repository cloned. Installing core dependencies...")

# Install PyTorch with compatible torchvision
!pip install torch==2.5.1+cu121 torchaudio==2.5.1+cu121 torchvision==0.20.1+cu121 --index-url https://download.pytorch.org/whl/cu121 -q

# Install your Colab-compatible chatterbox fork
!pip install git+https://github.com/devnen/chatterbox.git -q

print("✅ Core TTS components installed. Installing server dependencies...")

# Install essential server requirements (skip problematic packages)
!pip install fastapi uvicorn pyyaml soundfile librosa safetensors -q
!pip install python-multipart requests jinja2 watchdog aiofiles unidecode inflect tqdm -q
!pip install pydub audiotsm -q

# Try to install parselmouth (may fail on some systems)
!pip install parselmouth -q || echo "Parselmouth installation failed - unvoiced segment removal will be disabled"

print("✅ Installation complete! Some optional packages may have been skipped.")

Files removed: 0
Cloning into 'Chatterbox-TTS-Server'...
remote: Enumerating objects: 302, done.
remote: Counting objects: 100% (163/163), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 302 (delta 138), reused 119 (delta 119), pack-reused 139 (from 1)
Receiving objects: 100% (302/302), 18.94 MiB | 17.04 MiB/s, done.
Resolving deltas: 100% (147/147), done.
/content/Chatterbox-TTS-Server
✅ Repository cloned. Installing core dependencies...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.4/780.4 MB 798.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 96.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 130.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 122.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━ 289.3/664

In [ ]:
import torch
import sys
import torchaudio as ta
from chatterbox.tts import ChatterboxTTS
from chatterbox.mtl_tts import ChatterboxMultilingualTTS
import re

# Il tuo monkey patch per torch.load
original_torch_load = torch.load

def patched_torch_load(f, map_location=None, **kwargs):
    if map_location is None:
        map_location = 'cpu'
    return original_torch_load(f, map_location=map_location, **kwargs)

torch.load = patched_torch_load
if 'torch' in sys.modules:
    sys.modules['torch'].load = patched_torch_load

device = "cpu"
multilingual_model = ChatterboxMultilingualTTS.from_pretrained(device=device)

# Funzione per dividere il dialogo
def parse_dialogue(text):
    """
    Divide il testo del dialogo in speaker e battute
    """
    # Pattern per riconoscere i cambi di speaker (assumendo punteggiatura come indicatore)
    sentences = re.split(r'[.!?]+', text.strip())

    # Lista per contenere i segmenti di dialogo
    dialogue_parts = []
    current_speaker = "speaker1"

    for i, sentence in enumerate(sentences):
        if sentence.strip():
            # Alterna tra speaker1 e speaker2 basandosi su parole chiave o punteggiatura
            if any(word in sentence.lower() for word in ["pronto", "buongiorno", "sì", "perfetto", "bene"]):
                if i % 2 == 0:
                    current_speaker = "speaker1"  # Operatore CUP
                else:
                    current_speaker = "speaker2"  # Cliente
            else:
                # Alterna automaticamente
                current_speaker = "speaker1" if i % 2 == 0 else "speaker2"

            dialogue_parts.append({
                'speaker': current_speaker,
                'text': sentence.strip() + "."
            })

    return dialogue_parts

# Metodo migliorato: divisione manuale del dialogo
def create_manual_dialogue():
    """
    Crea manualmente la divisione del dialogo per maggiore controllo
    """
    dialogue = [
        {"speaker": "operator", "text": "Pronto, CUP, buongiorno come posso aiutarla?"},
        {"speaker": "client", "text": "Buongiorno, senta io avrei bisogno di prenotare una visita neurologica."},
        {"speaker": "operator", "text": "Certo, mi può dire il suo nome e cognome per favore?"},
        {"speaker": "client", "text": "Sì, sono Mario Bianchi."},
        {"speaker": "operator", "text": "Perfetto signor Bianchi, ora mi serve il suo codice fiscale."},
        {"speaker": "client", "text": "Allora... B-I-A-N-C-O... no scusi, B-I-A-N-C-H-I come Bari, M-A-R-I-O come Milano, settantadue... aspetti che controllo... settantadue, A come Ancona, zero-uno, L come Livorno, seicentoventisette, G come Genova."},
        {"speaker": "operator", "text": "Quindi BNCMRA72A01L627G, corretto?"},
        {"speaker": "client", "text": "Sì esatto."},
        {"speaker": "operator", "text": "Bene, per quale motivo ha bisogno della visita neurologica? Ha una impegnativa del medico di base?"},
        {"speaker": "client", "text": "Sì ce l'ho, è per dei mal di testa ricorrenti che ho da qualche mese, il dottore mi ha consigliato di fare una visita di controllo."},
        # Continua con il resto del dialogo...
    ]
    return dialogue

# Genera la conversazione
def generate_conversation():
    """
    Genera l'audio della conversazione con due voci diverse
    """
    dialogue = create_manual_dialogue()

    # Path per i file audio di riferimento (voci diverse)
    OPERATOR_VOICE = "registrazione.wav"  # Voce dell'operatore
    CLIENT_VOICE = "registrazione.wav"    # Stessa voce per ora, puoi cambiare

    audio_segments = []

    for i, turn in enumerate(dialogue):
        print(f"Generando: {turn['speaker']} - {turn['text'][:50]}...")

        # Scegli il prompt audio basandoti sul speaker
        if turn['speaker'] == "operator":
            audio_prompt = OPERATOR_VOICE
        else:
            audio_prompt = CLIENT_VOICE

        # Genera l'audio per questa battuta
        wav = multilingual_model.generate(
            turn['text'],
            audio_prompt_path=audio_prompt,
            language_id="it"
        )

        # Salva il segmento individuale
        segment_filename = f"segment_{i:02d}_{turn['speaker']}.wav"
        ta.save(segment_filename, wav, multilingual_model.sr)
        audio_segments.append(wav)

    return audio_segments

# Funzione per concatenare i segmenti audio
def concatenate_audio_segments(segments):
    """
    Concatena tutti i segmenti audio in un unico file
    """
    import torch

    # Aggiungi pause tra i segmenti
    pause_duration = int(0.5 * multilingual_model.sr)  # 0.5 secondi di pausa
    pause = torch.zeros(1, pause_duration)

    full_audio = []
    for i, segment in enumerate(segments):
        full_audio.append(segment)
        if i < len(segments) - 1:  # Non aggiungere pausa dopo l'ultimo segmento
            full_audio.append(pause)

    # Concatena tutto
    final_audio = torch.cat(full_audio, dim=1)
    return final_audio

# Esegui la generazione
if __name__ == "__main__":
    print("Generando conversazione...")
    segments = generate_conversation()

    print("Concatenando segmenti...")
    final_conversation = concatenate_audio_segments(segments)

    print("Salvando conversazione completa...")
    ta.save("conversazione_completa.wav", final_conversation, multilingual_model.sr)
    print("Conversazione salvata come 'conversazione_completa.wav'")
